# dataloaders.py

In [1]:
import sys
sys.path.append("../src")

from dataloaders import encode_label, decode_label



In [2]:
unique_classes = ["a", "b", "c"]
classes = ["a", "b"]

encoded = encode_label(classes, unique_classes)
print(encoded)


unique_classes = ["a", "b", "c"]
classes = ["a", "c"]

encoded = encode_label(classes, unique_classes)
print(encoded)

unique_classes = ["a", "b", "c"]
classes = [""]

encoded = encode_label(classes, unique_classes)
print(encoded)

[1, 1, 0]
[1, 0, 1]
[0, 0, 0]


In [3]:
unique_classes = ["a", "b", "c"]


decoded = decode_label([1, 0, 0], unique_classes)
print(decoded)


decoded = decode_label([1, 1, 0], unique_classes)
print(decoded)


decoded = decode_label([0, 0, 0], unique_classes)
print(decoded)

['a']
['a', 'b']
['']


In [4]:
import sys
sys.path.append("../src")

from dataloaders import ECGDataset

# Example of usage
path = '../data/physionet.org/files/ptb-xl/1.0.2/'
sampling_rate = 100  # or other value depending on your data

dataset = ECGDataset(path, sampling_rate)


[INFO] Loading data...
[INFO] Obtaining diagnostic_superclass ...


In [5]:
print(len(dataset))
print(dataset[0])


17420
{'x': tensor([[-1.1054, -0.6696,  1.1030,  ..., -0.2296, -0.4310, -0.7831],
        [-1.0778, -0.6215,  1.1200,  ..., -0.2820, -0.3749, -0.7341],
        [-1.1145, -0.5374,  1.3073,  ..., -0.2506, -0.3189, -0.6850],
        ...,
        [ 0.6191, -0.0087, -1.1616,  ...,  0.2944, -0.4534, -0.5771],
        [ 0.7750,  0.0394, -1.3660,  ...,  2.5792, -0.5094, -0.9695],
        [ 0.1880, -0.3812, -0.9062,  ...,  1.5416, -0.3861, -1.1853]]), 'y_decoded': ['NORM'], 'y': tensor([0, 0, 0, 0, 0, 1])}


In [6]:
dataset[0]['x'].shape, dataset[0]['y'].shape

(torch.Size([1000, 12]), torch.Size([6]))

In [7]:
from torch.utils.data import DataLoader


# Create a DataLoader
batch_size = 32


dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Using the DataLoader in a training loop
for batch in dataloader:
    print(batch['x'].shape, batch['y'].shape)
    print(batch['y'][0])
    break

torch.Size([32, 1000, 12]) torch.Size([32, 6])
tensor([0, 0, 1, 0, 0, 0])


# models.py

In [1]:
import sys
sys.path.append("../src")


from models import SinusoidalPosEmb




In [3]:
import torch

pos_emb = SinusoidalPosEmb(dim=16)

x = torch.randn(1, 1000)
print(pos_emb(x).shape)

x = torch.randn(1, 1000, 12)
y = pos_emb(x) # (B, T, C) -> (B, T, C, 16)
print(y.shape)
# convert to (B, T, 16*C)
y = y.flatten(-2)
print(y.shape)

# 16*12 -> 192


torch.Size([1, 1000, 16])
torch.Size([1, 1000, 12, 16])
torch.Size([1, 1000, 192])


In [4]:
import sys
sys.path.append("../src")
from models import Extractor
import torch

extractor = Extractor(dim_base=128, dim=384)

x = torch.randn(2, 1000, 12)
y = extractor(x, Lmax=128)
print(y.shape)


627 755
torch.Size([2, 128, 384])


In [2]:
import sys
sys.path.append("../src")
from models import DeepHeartModel
import torch

model = DeepHeartModel()

x = torch.randn(2, 512, 12)
y = model(x)
print(y.shape)


torch.Size([2, 5])


In [3]:
%timeit y = model(x)

129 ms ± 5.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
